In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from flask import Flask

In [2]:
# acquire full html contents to search through
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
# response.text
soup = BeautifulSoup(response.text,'html.parser')
# print(soup.prettify())

In [3]:
#find all <div class=xx>, then take only the latest result + title, paragraph text
title = soup.find_all('div',class_='content_title')[0].text
# print(title)
paragraph = soup.find_all('div',class_='rollover_description_inner')[0].text
# print(paragraph)

In [4]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
#open Splinter
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\benja\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


In [6]:
html = browser.html
soup = BeautifulSoup(response.text,'html.parser')

### I can't get this line to work but I'm pretty sure it's accurate
# grab image text (see following line) and remove unnecessary text before adding to final url
image_url = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
# background-image: url('  /spaceimages/images/wallpaper/PIA14293-1920x1200.jpg  ');
base_url = 'https://www.jpl.nasa.gov'
featured_image_url = base_url + image_url
print(featured_image_url)

KeyError: 'style'

In [7]:
url = 'https://space-facts.com/mars/'
#grab table
table = pd.read_html(url)
# table
#drop non-mars info
mars_table_df = table[0]
mars_table_html = mars_table_df.to_html(classes='table table-striped')
print(mars_table_html)

<table border="1" class="dataframe table table-striped">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th

In [8]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

hemispheres = soup.find_all('div', class_='item')
hemispheres_list = []
hemispheres
base_url = 'https://astrogeology.usgs.gov'

for data in hemispheres:
    title = data.find('h3').text
    #link to hemisphere page
    hemisphere_url = data.find('a', class_='itemLink product-item')['href']
    browser.visit(base_url + hemisphere_url)
    #on next page, grab image url and make full url link
    page2_html = browser.html
    soup = BeautifulSoup(page2_html, 'html.parser')
    image_url = soup.find('img',class_='wide-image')['src']
    full_image_url = base_url + image_url
    #add to list
    hemispheres_list.append({"title":title,"img_url":full_image_url})

hemispheres_list

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\benja\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]